In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os

os.chdir('C:\\Users\\PC\\Downloads')

In [2]:
train = pd.read_csv("Train.txt",sep = "#|;",engine = 'python',header = None)
test = pd.read_csv("Test.txt",sep = "#|;",engine = 'python',header = None)
cross_validation = pd.read_csv("Cross_Validation.txt",sep = "#|;",engine = 'python',header = None)

In [3]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.model_selection import KFold,GroupKFold,StratifiedGroupKFold
from sklearn.ensemble import AdaBoostClassifier,StackingClassifier,BaggingClassifier
from sklearn.utils import shuffle

In [4]:
train

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
0,0,-6.024003,8.434972,-1.653654,-0.574355,-2.350995,0.785241,-2.032504,-1.689566,-0.503721,...,3.419179,2.334640,-0.974489,0.695670,-3.212498,-0.472166,1.425531,0.963218,0.243333,5.883590
1,0,-13.949878,9.602716,1.763657,4.703395,-1.207065,0.873615,1.585900,-2.387653,-3.466051,...,-3.674066,-4.262962,-3.463864,-2.229005,-3.212898,-1.256255,-2.589354,-1.541394,-2.369498,1.900734
2,0,5.170417,2.401173,6.257058,1.202883,1.516425,-1.410218,-4.488354,-1.157383,1.112390,...,-5.495467,0.622731,-0.687237,1.318741,-1.138520,-2.500563,-0.281380,1.664352,1.963432,4.670355
3,0,-3.411793,11.979136,0.022563,0.797471,-2.327754,-3.992630,2.597402,-1.321857,1.213038,...,-1.263155,-5.289630,-1.436241,-4.418044,-3.450107,-1.565164,1.307567,0.455341,-0.370953,4.833021
4,0,-8.137907,16.958149,0.525468,-0.518099,0.318236,-1.400949,-1.171182,-0.158977,-1.085432,...,-5.699882,-4.840200,-2.620963,-0.714717,-1.426346,1.110394,2.860897,1.772555,1.419513,4.536200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18165,23,-20.270353,1.089628,-3.105801,-2.609111,-2.497966,-4.682183,0.509501,-0.511175,2.287552,...,2.988654,-1.562807,-6.267464,0.333675,1.501495,1.040096,-3.691595,0.407229,0.095279,2.750888
18166,23,-15.053128,-2.819717,-0.884350,-2.045222,1.609190,-2.138863,0.775223,1.138954,-0.206750,...,-3.805477,0.334937,2.721093,-3.690767,1.325286,2.754217,-0.306933,-2.612856,-0.956152,3.460388
18167,23,-20.249252,2.040117,-1.584354,-4.785700,-1.209069,-6.339949,0.402131,-1.203664,0.005176,...,-3.800741,3.777278,-3.952350,-3.115205,-2.404258,0.414806,-0.902465,-0.838150,-1.429881,1.083172
18168,23,8.053919,-3.056077,5.910312,-3.730439,-0.544434,3.621176,-2.932716,-4.207619,1.945056,...,4.062116,4.873626,-1.237416,-1.985327,-1.190132,2.142913,-1.322156,0.048656,0.282358,4.390202


In [5]:
test

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
0,0,-4.835846,11.485481,2.063461,2.515862,-1.508640,-4.832501,-0.874070,-0.216661,0.839478,...,0.222955,1.100960,1.050461,-0.543098,1.471425,0.195184,1.142715,-0.177082,0.534256,3.988318
1,0,-9.180304,4.764287,4.573371,2.262439,-0.830693,-3.553008,0.759175,-0.869035,1.169368,...,2.261477,2.245412,2.951444,1.916286,0.110373,0.570941,0.024815,-0.169944,-0.248919,5.425950
2,0,4.224361,10.227818,4.034547,2.664427,2.017728,1.051023,-3.101555,-2.001828,0.092069,...,-8.855023,-6.830781,3.879649,0.278574,-1.238731,-1.479367,-1.186802,4.526675,0.879041,4.828293
3,0,5.062918,3.672025,7.242286,2.297312,1.442639,-3.738971,-0.653979,0.762901,-0.514581,...,-6.429011,2.002834,5.006701,2.745507,-1.306818,2.183330,0.693087,0.500900,-0.435711,-0.128711
4,0,1.483761,6.360590,10.521129,3.592051,1.969661,-0.720841,-2.794724,-2.613616,0.713393,...,-6.423320,0.111574,1.210618,4.056096,1.017101,-0.136407,-1.178811,0.623710,-0.047168,3.526209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,23,-9.413824,-2.825387,1.389150,-1.344054,5.035080,0.838401,-0.190306,1.885224,-0.952709,...,-0.459922,-3.211695,-0.771446,1.289810,0.355308,-1.646043,-0.334202,0.895260,0.587216,3.464514
1005,23,-11.194378,-6.600467,-1.546185,-2.049784,2.214451,-4.067553,3.456781,0.432913,1.872509,...,-3.629748,-1.742132,1.373383,-0.601796,3.510332,1.140225,-3.606575,-0.525001,3.638772,2.789713
1006,23,-16.711020,-5.274776,-0.905376,-6.061351,2.096176,0.236713,1.822124,4.087193,-0.023794,...,0.444186,0.929874,1.427553,1.217588,-0.136986,1.564861,1.790452,0.264197,-0.664185,4.595086
1007,23,-18.062640,1.872057,-1.302444,-0.409882,-1.109246,-0.646558,2.272523,-0.476603,0.724692,...,-0.260455,-1.667005,4.141400,2.848016,-1.358906,0.070012,1.436267,-1.582356,-1.177407,4.069070


In [6]:
cross_validation

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
0,0,-10.262328,10.223884,6.260454,-0.564348,6.744207,-2.798208,-0.200011,-1.562980,-1.088669,...,-6.887674,-2.767490,-0.796527,-2.232239,-0.212699,-0.472747,-4.029301,-0.349139,0.696110,4.805770
1,0,-26.083918,-0.192849,7.422371,-4.705012,4.875607,0.127555,-0.856406,2.121985,-2.523355,...,-2.181474,2.706180,2.003653,2.737074,1.428900,0.180737,-0.582639,1.616664,1.460235,2.698359
2,0,-10.691353,14.242621,3.272347,3.471687,3.535090,-1.427409,-2.125192,-1.270264,-1.447249,...,-4.459030,-3.959104,0.092738,2.027900,0.200879,0.474645,-1.640315,0.408943,0.323539,3.087251
3,0,-25.717509,0.938196,-2.343231,3.337248,-2.388594,-0.183610,1.103012,1.519615,1.631139,...,-0.614165,0.580259,3.339510,1.230932,-3.043870,0.549192,0.129573,-0.207652,1.898810,5.287238
4,0,-26.380413,10.401352,3.809617,3.378589,2.773404,-0.435435,0.842176,-2.388515,0.992311,...,1.358296,5.335774,5.153475,3.690575,1.463203,-0.750382,-0.370209,1.064637,1.333647,4.794315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,23,-6.562270,-4.111161,2.029770,-3.544560,1.470645,0.475510,-2.191993,-0.531015,-1.234192,...,-2.222540,-2.550281,-0.686531,-2.781893,-3.453743,0.692056,2.753201,2.146726,-1.538186,4.723725
1006,23,-15.976607,-0.298060,-3.988413,-0.741452,-1.654719,-2.355983,4.889074,1.337285,1.049679,...,-1.511561,-6.298859,-0.359862,1.220160,-2.636720,-1.772738,0.534132,-0.662344,-1.413739,4.867703
1007,23,0.732383,2.798996,1.116006,-2.945275,1.883308,-1.639882,-0.459453,-0.067911,-0.090070,...,-0.252409,-0.764838,-1.898047,1.163950,-0.526930,1.777166,0.804445,-1.687606,-0.779547,5.338880
1008,23,-14.056767,4.379005,-3.742948,-0.562560,-1.330001,2.850805,-1.458133,1.041708,-0.434027,...,-2.054648,-5.095892,2.271301,-0.103916,-3.253677,-1.953994,-1.010396,-0.767318,-1.342337,3.357120


In [7]:
data=shuffle(pd.concat([train,test,cross_validation]))
data.reset_index(inplace=True)
data.drop("index",axis=1,inplace=True)

In [8]:
data

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
0,7,-21.909266,7.047152,0.247626,0.480946,-5.747605,4.113030,-1.481534,-0.214033,-1.374867,...,1.432681,-4.262781,-6.129777,-3.906247,-3.638932,-4.760786,-3.992896,-3.777917,-3.780022,3.322974
1,23,-1.908556,-2.101860,1.427560,-5.790903,1.911884,0.574645,-0.549089,0.756202,1.665348,...,0.803396,3.229471,-3.118001,-1.604494,2.928705,1.766971,-0.251061,2.327928,-0.979660,2.104203
2,6,-28.060449,4.057583,-2.941313,2.058098,-0.700173,-1.425173,-1.168267,1.720342,1.788379,...,-1.004054,-2.751202,1.663985,5.094221,0.489655,0.940730,-0.257755,1.272641,0.544617,3.152345
3,3,-7.528496,-5.362270,3.804232,-4.480017,0.522330,-1.845216,-3.920646,0.484953,0.070714,...,1.174197,-2.095006,-2.116396,0.449771,1.050395,2.822405,1.101954,0.737784,3.033181,4.723774
4,21,-3.365590,1.507412,7.200625,-0.566479,-0.215611,-3.644524,-2.964181,-0.713593,2.512834,...,-8.670554,0.217568,-2.890368,-3.315098,0.246156,-2.217732,1.632700,0.475211,-0.965897,3.687315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20184,11,-21.515869,-5.210914,4.995179,-2.926548,-2.254311,-3.426687,2.418002,-3.094484,-0.773098,...,1.529647,3.928154,-1.017476,1.991962,-0.351281,-0.624191,0.546957,1.228110,0.700929,4.308911
20185,18,17.319472,-7.863206,6.296696,0.533821,-8.171208,-1.133855,-1.636093,1.746683,-4.009881,...,0.769987,2.906721,3.499198,1.167008,1.950961,-1.554756,0.845545,-1.562137,-0.256899,0.263249
20186,12,-3.212212,1.763779,-15.163487,-1.013287,1.722790,-0.506251,0.423653,-1.425959,-1.496100,...,-1.070632,0.393236,-3.708068,-2.972350,-0.798867,0.249084,2.350901,0.656930,1.565987,4.262493
20187,21,-18.881575,-7.802555,4.363258,-2.982100,1.721635,-1.532089,-6.113338,0.061432,2.159759,...,-4.337708,1.972270,-5.300464,-0.226944,-1.544544,0.107174,1.433571,-1.684138,0.984786,4.302194


In [9]:
data.isnull().sum().sort_values(ascending=False)

0      0
367    0
361    0
362    0
363    0
      ..
176    0
175    0
174    0
173    0
546    0
Length: 547, dtype: int64

In [10]:
data.isna().sum().sort_values(ascending = False)

0      0
367    0
361    0
362    0
363    0
      ..
176    0
175    0
174    0
173    0
546    0
Length: 547, dtype: int64

In [11]:
data[0].unique()

array([ 7, 23,  6,  3, 21, 20,  9,  4, 15, 19, 22, 17,  1,  5,  2, 16, 12,
       18, 14, 11, 13,  8, 10,  0], dtype=int64)

In [12]:
scaler=MinMaxScaler()
X=data.drop(0,axis=1)
X=scaler.fit_transform(X)

Y=data[0]

In [13]:
X_train,X_test,Y_train,Y_test,=train_test_split(X,Y,train_size=0.85)

# 

In [14]:
def find_best_model(learning_algorithm, X, Y, expected_accuracy, 
                    model_selection=None, model_selection_params={}, params={},
                    max_iterations=20):

    import pickle
    from copy import deepcopy
    
    iteration = 0
    best_model = None
    best_accuracy = 0.0
    
    while iteration < max_iterations:
        train_size = np.round(np.random.uniform(0.6, 0.95), 2)
        X_train, X_val, y_train, y_val = train_test_split(X, Y, train_size=train_size)
        
        if model_selection is not None:
            params_copy = deepcopy(params)
            for key, value in params.items():
                model = learning_algorithm()
                if not isinstance(value, (list, tuple)):
                    value = [value]
                value.append(model.__dict__.get(key))
                params_copy[key] = list(set(value))
            
            selected_model = model_selection(learning_algorithm(), params_copy, scoring='accuracy', **model_selection_params)
            model = selected_model.fit(X_train, y_train)
        else:
            model = learning_algorithm(**params)
            model.fit(X_train, y_train)
        
        y_val_predict = model.predict(X_val)
        test_accuracy = accuracy_score(y_val_predict, y_val)
        
        print(f"Train Size: {train_size:.2f}, Model: {learning_algorithm.__name__}, Test Accuracy:{test_accuracy:.3f}")
        
        if test_accuracy >= expected_accuracy and test_accuracy > best_accuracy:
            best_model = model
            best_accuracy = test_accuracy
            
            if os.path.exists(f"{learning_algorithm.__name__.lower()}1.pkl"):
                os.remove(f"{learning_algorithm.__name__.lower()}1.pkl")
                
            with open(f"{learning_algorithm.__name__.lower()}1.pkl", "wb") as file:
                pickle.dump(best_model, file)
        
        iteration += 1
    
    if best_model is None:
        raise Exception(f"Could not achieve expected accuracy of {expected_accuracy} in {max_iterations} iterations.")
    
    return best_accuracy, best_model


# 

In [15]:
tree = DecisionTreeClassifier()
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,train_size=0.90)
tree.fit(x_train,y_train)

accuracy_score(y_val,tree.predict(x_val))

0.7558275058275058

In [16]:
accuracy,model=find_best_model(DecisionTreeClassifier,X_train,Y_train,expected_accuracy=0.78)

Train Size: 0.64, Model: DecisionTreeClassifier, Test Accuracy:0.732
Train Size: 0.78, Model: DecisionTreeClassifier, Test Accuracy:0.756
Train Size: 0.80, Model: DecisionTreeClassifier, Test Accuracy:0.768
Train Size: 0.92, Model: DecisionTreeClassifier, Test Accuracy:0.763
Train Size: 0.80, Model: DecisionTreeClassifier, Test Accuracy:0.771
Train Size: 0.67, Model: DecisionTreeClassifier, Test Accuracy:0.761
Train Size: 0.75, Model: DecisionTreeClassifier, Test Accuracy:0.757
Train Size: 0.83, Model: DecisionTreeClassifier, Test Accuracy:0.758
Train Size: 0.76, Model: DecisionTreeClassifier, Test Accuracy:0.757
Train Size: 0.64, Model: DecisionTreeClassifier, Test Accuracy:0.735
Train Size: 0.93, Model: DecisionTreeClassifier, Test Accuracy:0.779
Train Size: 0.72, Model: DecisionTreeClassifier, Test Accuracy:0.751
Train Size: 0.65, Model: DecisionTreeClassifier, Test Accuracy:0.736
Train Size: 0.65, Model: DecisionTreeClassifier, Test Accuracy:0.749
Train Size: 0.94, Model: DecisionT

# 

In [17]:
lg = LogisticRegression()
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,train_size=0.90)
lg.fit(x_train,y_train)

accuracy_score(y_val,lg.predict(x_val))

0.9347319347319347

In [18]:
params={"solver":['sag', 'saga'],"max_iter":[400,500],"n_jobs":4}
accuracy_log , model_log = find_best_model(LogisticRegression,X_train,Y_train,expected_accuracy=0.94,params={"solver":"saga"})

Train Size: 0.85, Model: LogisticRegression, Test Accuracy:0.929
Train Size: 0.68, Model: LogisticRegression, Test Accuracy:0.929
Train Size: 0.70, Model: LogisticRegression, Test Accuracy:0.925
Train Size: 0.74, Model: LogisticRegression, Test Accuracy:0.931
Train Size: 0.71, Model: LogisticRegression, Test Accuracy:0.940
Train Size: 0.70, Model: LogisticRegression, Test Accuracy:0.933
Train Size: 0.81, Model: LogisticRegression, Test Accuracy:0.933
Train Size: 0.91, Model: LogisticRegression, Test Accuracy:0.943
Train Size: 0.86, Model: LogisticRegression, Test Accuracy:0.937
Train Size: 0.95, Model: LogisticRegression, Test Accuracy:0.921
Train Size: 0.75, Model: LogisticRegression, Test Accuracy:0.933
Train Size: 0.93, Model: LogisticRegression, Test Accuracy:0.928
Train Size: 0.86, Model: LogisticRegression, Test Accuracy:0.940
Train Size: 0.82, Model: LogisticRegression, Test Accuracy:0.933
Train Size: 0.93, Model: LogisticRegression, Test Accuracy:0.929
Train Size: 0.67, Model: 